<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part One: Data Preparation </h2>	


<h3>  Import Statements </h3>	


In [1]:
#Import anything you need here
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import os


In [2]:
#Example of reading in a file

print(skimage.data_dir)

filename = os.path.join(skimage.data_dir, 'd.jpg')
dog = io.imread(filename)
skimage.io.imshow(dog)

plt.show()
filename2 = os.path.join(skimage.data_dir, 'c.jpg')
cat = io.imread(filename2)
example_data_frame= pd.DataFrame()


example_data_frame['Pictures']=[dog, cat]
example_data_frame['Encoding']=[1,2]
example_data_frame

C:\Users\Ashwin\Anaconda3\lib\site-packages\skimage\data


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Ashwin\\Anaconda3\\lib\\site-packages\\skimage\\data\\d.jpg'

<h4> Using the code for reference, take the given folder and create a dataframe with the picture object, and the encoding as listed below. </h4>	
0=Airplanes, 1=Bear, 2=Blimp, 3=Comet, 4=Crab, 5=Dog, 6=Dolphin, 7=Giraffe, 8=Goat, 9=Gorilla, 10=Kangaroo, 11=Killer-Whale, 12=Leopards, 13=Llama, 14= Penguin, 15= Porcupine, 16=Teddy-Bear, 17=Triceratops, 18=Unicorn, 19=Zebra

In [38]:
def read_organize_data(file_path):
    #Stores encodings
    #TODO: Resize Images to constant size? Some of them are very lopsided
    category_encodings = {'airplanes': 0, 'bear': 1, 'blimp': 2, 'comet': 3, 'crab': 4, 'dog': 5, 
                'dolphin': 6, 'giraffe': 7, 'goat': 8, 'gorilla': 9, 'kangaroo': 10, 
                'killer-whale': 11, 'leopards': 12, 'llama': 13, 'penguin': 14, 'porcupine': 15,
                'teddy-bear': 16, 'triceratops': 17, 'unicorn': 18, 'zebra': 19}
    
    image_df = pd.DataFrame(columns = ['Pictures', 'Encoding'])
    data, encodings = [], []
    for category in os.listdir(file_path):
        if category[0] != '.': #Protecting against temporary files (Windows fix)
            for file_name in os.listdir(file_path + category):
                if file_name[0] != '.':
                    file_encode = category_encodings[category]
                    file_data = io.imread(file_path + category + '/' + file_name)
                    encodings.append(file_encode)
                    data.append(file_data)
    image_df['Pictures'] = data
    image_df['Encoding'] = encodings
    return image_df
    

In [39]:
starting_data = read_organize_data("20_categories_training/")